# Aniruddha Jain

# G21548859

# Homework 2

## Due Wednesday February 22, 3:30PM

Before we start, make sure you have installed using "pip install" the following libraries:
1. pip install beautifulsoup4
2. pip install requests
3. pip install lxml
4. pip install yfinance


In [99]:
import requests
import lxml
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup


In [100]:
# !pip install yfinance 
import yfinance as yf
import pandas as pd

In [103]:
#Get soup for the search results
html_data=requests.get('https://www.timesjobs.com/candidate/job-search.html?searchType=personalizedSearch&from=submit&txtKeywords=python&txtLocation=').text
soup=BeautifulSoup(html_data,'lxml')

#Search for all the job cards
jobs=soup.find_all('li', class_='clearfix job-bx wht-shd-bx')

#Within each job we want to extract the link that gives us the "More Info"
#This will be contained in the 'a' tag in the 'href' value
#We did this in class for the first job

moreinfolinks=[]
AllInfo=pd.DataFrame()

for onejob in jobs:
    company_name=onejob.find('h3', class_='joblist-comp-name').text.strip()
    moreinfolink=onejob.header.h2.a['href']
    moreinfolinks.append(moreinfolink) #Store all the moreinfo links in case you need to access it later
    
    #Extract the soup for the "more info" page
    moreinfopage=requests.get(moreinfolink)
    moreinfosoup=BeautifulSoup(moreinfopage.text,'lxml')
    jobdesc=moreinfosoup.find('div', class_='jd-desc job-description-main').text
    
    #Clean up the description (NOT required but makes it look nice and readable)
    jobdesc=jobdesc.replace("Job Description", "")
    jobdesc=jobdesc.strip().replace("<br/>",", ")
    
    #Create an empty row which will contain all the information we collect
    #We will then write this row to a csv file (or a Pandas DataFrame)
    row=[]
    row.append(company_name)
    row.append(jobdesc)

    #Extract other job info
    otherinfo=moreinfosoup.find('div', class_='job-basic-info')
    funclist=otherinfo.find_all('li', class_='clearfix')
    for f in funclist:
        
        #Grab the label (i.e. "Industry", "Specialization", etc.)
        label=f.label.text[:-1]
        #Employment Type is the only label whose text is not in a <span> class
        if label=="Employment Type":
            info=f.text.replace("Employment Type:", "").strip()
            row.append(info)
        elif label!="Role":
            #I didn't realize two job postings had an additional row for "Role", can skip it
            info=f.span.text.strip()
            row.append(info)
        
    
    #Write all that to a new row in the Pandas DataFrame we initialized earlier
    AllInfo=AllInfo.append(pd.Series(row), ignore_index=True)

AllInfo.columns=["Company", "Job Description", "Job Function", "Industry", "Specialization", "Qualification", "Employment Type"]



/var/folders/v4/pcb5pkfn5h7b0vj_np_0db040000gn/T/ipykernel_72199/462809024.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  AllInfo=AllInfo.append(pd.Series(row), ignore_index=True)


In [104]:
AllInfo

,Company,Job Description,Job Function,Industry,Specialization,Qualification,Employment Type
0,Radical Tecnologies\r\n (More Jobs),We are looking for PYTHON Online Trainer Part...,IT Software : Software Products & Services,IT-Software,"Application Programming,Client Server,Configur...",Any Graduate,Part Time
1,Surya Informatics Solutions Pvt. Ltd.,POSITION: Python Developer ELIGIBILITY: FRESHE...,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,Any Graduate,Full Time
2,learntek\r\n (More Jobs),Greetings from Learntek!!\n\nWe have Urgent Re...,"Teaching/Education,HR/PM/IR/Training","Educational/Training, IT-Software","Private Tuition,Special Education Teaching,Onl...",Any Graduate,Part Time
3,ERM Placement Services Pvt. Ltd.\r\n (More...,Role Python Developer\nSolid experience in Pyt...,IT Software : Software Products & Services,Consulting Services,Application Programming,BE/ B.Tech (Engineering),Full Time
4,Perfios Software,Roles and Responsibilities: Exaperience: 5-12Y...,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,Any Graduate,Full Time
5,Ivan Infotech Pvt. Ltd.,We are looking for a Python Developer to join ...,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,Any Graduate,Full Time
6,Triadss Tech Solutions,:Work experience as a Python DeveloperExpertis...,IT Software : Software Products & Services,Consulting Services,Software Engineer,BE/ B.Tech (Engineering),Full Time
7,GEMINI SOFTWARE SOLUTIONS,Python Finance Python Finance5 Years Multiple...,IT Software : Software Products & Services,Others - other Industry,Software Engineer,Any Graduate,Full Time
8,east india securities ltd.,job_description 2 years of experience worki...,IT Software : Software Products & Services,"Petroleum/Oil and Gas/Power, Construction/Cem...",Software Engineer,Any Graduate,Full Time
9,Techment Technology Inc.,Python Engineer We are looking for a Python De...,IT Software : Software Products & Services,"IT-Hardware/Networking, Telecom",Software Engineer,Any Graduate,Full Time


## Question 2

### Part A
- In Lecture 4 we extracted tables from the Yahoo Finance page for AAPL
- Extract all the information from the tables from statistics page for AAPL
https://finance.yahoo.com/quote/AAPL/key-statistics?p=AAPL
- We only want the "Current" column from the main table and also all the other financial information on the page (like the Financial Highlights, Stock Price History, Profitability, Management Effectivess, etc.)
- The output must be *one* Pandas dataframe 
    - The index of the dataframe must be the variable name
    - The only column must contain the latest values
    - You don't need to worry about categorizing the variables into sub-categories like on the YF website (for example, "Return on Assets (ttm)" appears under the sub-category "Management Effectiveness". You only need to capture "Return on Assets (ttm)" and its value, not the title of the category "Management Effectiveness"

- Make sure to use the "headers" information in your requests.get() call like we did in Lecture 4

In [3]:
# Define the URL of the statistics page for AAPL
url = "https://finance.yahoo.com/quote/AAPL/key-statistics?p=AAPL"

# Send a GET request to the URL with headers
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)

# Create a BeautifulSoup object from the response
soup = BeautifulSoup(response.content, 'html.parser')


In [4]:
# Find the main table on the page
main_table = soup.find_all('table')[0]

# Find all the rows in the main table
rows = main_table.find_all('tr')

# Create a dictionary to store the extracted data
data_dict = {}

In [5]:
# Loop through the rows and extract the data for the "Current" column
for row in rows:
    cells = row.find_all('td')
    if len(cells) == 3:
        variable_name = cells[0].get_text()
        current_value = cells[2].get_text()
        data_dict[variable_name] = current_value


In [6]:
# Find all the tables on the page except for the main table
other_tables = soup.find_all('table')[1:]


In [7]:
# Loop through the other tables and extract the data
for table in other_tables:
    rows = table.find_all('tr')
    for row in rows:
        cells = row.find_all('td')
        if len(cells) == 2:
            variable_name = cells[0].get_text()
            current_value = cells[1].get_text()
            data_dict[variable_name] = current_value


In [8]:
# Create a Pandas dataframe from the extracted data
df = pd.DataFrame.from_dict(data_dict, orient='index', columns=['Latest Value'])


In [9]:
df

,Latest Value
Beta (5Y Monthly),1.30
52-Week Change 3,-7.59%
S&P500 52-Week Change 3,-10.33%
52 Week High 3,178.49
52 Week Low 3,124.17
50-Day Moving Average 3,151.46
200-Day Moving Average 3,148.50
Avg Vol (3 month) 3,68.88M
Avg Vol (10 day) 3,61.8M
Shares Outstanding 5,15.82B


### Part B
- Extend your code from Part A to be able to extract the same set of financial information from any number of tickers as provided by a list
- Save your data as *one* Pandas DataFrame
    - The index should be the Statistic and the name of the columns should be the ticker
- Please *DO NOT* try to run this script on more than a few tickers, you don't want to be flagged as a bot.
- Test your code on the following list of tickers:
"AMC", "MSFT", "AAPL", "TSLA", "XOM"


In [13]:
ticlist=["AMC", "MSFT", "AAPL", "TSLA", "XOM"]


# Extend your code from Part A to the ticlist above

# Go to each of these companies' Yahoo Finance summary page and
# extract the same set of tables as you did in Part A

# Output should be one Pandas DataFrame where the columns are the tickers and the rows are the Statistics


In [14]:
# Define an empty dictionary to store the extracted data for each ticker
data_dict = {}


In [15]:
# Loop through each ticker in the ticlist
for tic in ticlist:

    # Define the URL of the statistics page for the ticker
    url = f"https://finance.yahoo.com/quote/{tic}/key-statistics?p={tic}"

    # Send a GET request to the URL with headers
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)

    # Create a BeautifulSoup object from the response
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the main table on the page
    main_table = soup.find_all('table')[0]

    # Find all the rows in the main table
    rows = main_table.find_all('tr')

    # Create a dictionary to store the extracted data for this ticker
    tic_dict = {}
    
    # Loop through the rows and extract the data for the "Current" column
    for row in rows:
        cells = row.find_all('td')
        if len(cells) == 3:
            variable_name = cells[0].get_text()
            current_value = cells[2].get_text()
            tic_dict[variable_name] = current_value

    # Find all the tables on the page except for the main table
    other_tables = soup.find_all('table')[1:]
    # Loop through the other tables and extract the data
    for table in other_tables:
        rows = table.find_all('tr')
        for row in rows:
            cells = row.find_all('td')
            if len(cells) == 2:
                variable_name = cells[0].get_text()
                current_value = cells[1].get_text()
                tic_dict[variable_name] = current_value

    # Add the extracted data for this ticker to the data_dict
    data_dict[tic] = tic_dict


In [16]:
# Create a Pandas dataframe from the extracted data
df = pd.DataFrame.from_dict(data_dict, orient='index')


In [17]:
df

,Beta (5Y Monthly),52-Week Change 3,S&P500 52-Week Change 3,52 Week High 3,52 Week Low 3,50-Day Moving Average 3,200-Day Moving Average 3,Avg Vol (3 month) 3,Avg Vol (10 day) 3,Shares Outstanding 5,...,Diluted EPS (ttm),Quarterly Earnings Growth (yoy),Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Current Ratio (mrq),Book Value Per Share (mrq),Operating Cash Flow (ttm),Levered Free Cash Flow (ttm)
AMC,2.04,-78.51%,-10.33%,27.50,3.77,5.45,8.71,35.49M,27.71M,513.33M,...,-2.05,N/A,631.5M,1.22,10.02B,N/A,0.53,-5.08,-628.5M,-291.58M
MSFT,0.92,-8.47%,-10.33%,315.11,213.43,260.43,253.25,32.73M,30.93M,7.44B,...,8.99,-12.50%,99.5B,13.37,77.98B,42.58,1.93,24.59,84.39B,44.61B
AAPL,1.30,-7.59%,-10.33%,178.49,124.17,151.46,148.50,68.88M,61.8M,15.82B,...,5.88,-13.40%,51.36B,3.25,111.11B,195.87,0.94,3.58,109.19B,84.73B
TSLA,2.07,-45.67%,-10.33%,384.29,101.81,187.51,215.82,174.72M,131.72M,3.16B,...,3.84,58.90%,22.19B,7.01,5.75B,12.52,1.53,14.13,14.72B,4.21B
XOM,1.08,31.87%,-10.33%,119.63,79.29,110.67,102.14,16.99M,17.24M,4.07B,...,13.26,43.70%,29.64B,7.28,46.92B,23.17,1.41,47.78,76.8B,46.27B


# Question 2
- In Lecture 5, we were able to calculate the mean and standard deviation of a portfolio that invested 50% of its money in AAPL and 50% of its money in SBUX.
- Imagine an investor in Jan 2018 with $1000 and wants to put 25% of their money in Disney (DIS), 50% of money in Netflix (NFLX), and the rest in Tesla (TSLA).
- Download the daily prices of DIS, NFLX, and TSLA from yfinance from 1/1/2018 to 12/31/2022
- Resample the data to a monthly frequency (keeping the last trading day per month)
- How many shares of each firm would the investor buy in January 2018?
- What would have been the average *monthly* return to the strategy employed by the investor over this time period?
- What would have been the standard deviation of *monthly* returns to the strategy employed by the investor over this time period?
- IMPORTANT NOTE: I am *not* asking you to derive a mathematical solution since we did not go over a 3-stock portfolio in class. I *am* asking you to follow the same steps we took for the 50/50 portfolio of AAPL and SBUX, and calculate the mean and standard deviation based on manually tracking the price changes in the portfolio over 5 years.

- Download the daily prices of DIS, NFLX, and TSLA from yfinance from 9/29/2016 to 9/29/2021

In [20]:
tickers=['DIS','NFLX','TSLA']
# start='9/29/2016'
# end='9/29/2021'
data = yf.download(tickers, start='2016-9-29', end='2021-9-29')['Close']
#data.reset_index(inplace=True) 
# data

[*********************100%***********************]  3 of 3 completed


In [21]:
data.dropna()

,DIS,NFLX,TSLA
Date,,,
2016-09-29,91.800003,96.669998,13.380000
2016-09-30,92.860001,98.550003,13.602000
2016-10-03,92.489998,102.629997,14.246667
2016-10-04,92.589996,102.339996,14.094000
2016-10-05,92.449997,106.279999,13.897333
...,...,...,...
2021-09-22,173.649994,590.650024,250.646667
2021-09-23,176.250000,593.260010,251.213333
2021-09-24,176.000000,592.390015,258.130005


In [22]:
duplicate=data.duplicated()
data[duplicate]

,DIS,NFLX,TSLA
Date,,,


- Resample to a monthly frequency

In [23]:
monthly_data = data.resample('M').last()

In [24]:
monthly_data

,DIS,NFLX,TSLA
Date,,,
2016-09-30,92.860001,98.550003,13.602000
2016-10-31,92.690002,124.870003,13.182000
2016-11-30,99.120003,117.000000,12.626667
2016-12-31,104.220001,123.800003,14.246000
2017-01-31,110.650002,140.710007,16.795334
...,...,...,...
2021-05-31,178.649994,502.809998,208.406662
2021-06-30,175.770004,528.210022,226.566666
2021-07-31,176.020004,517.570007,229.066666


In [25]:
# eomdata.shape

- How many shares of each firm would the investor buy in September 2016? (Remember the investor is taking $1000 and putting 25% in Disney (DIS), 50% in Netflix (NFLX), and the rest in Tesla (TSLA).

- Track how the portfolio would do in each month from September 2016 to September 2021

In [27]:
weights = [0.25, 0.5, 0.25]

prices = data.loc['2016-09-30', tickers]

total_investment = 1000

investments = [weight * total_investment for weight in weights]

shares = [investment / price for investment, price in zip(investments, prices)]



In [28]:
for ticker, share_count in zip(tickers, shares):
    print(f"Shares of {ticker} to buy: {share_count}")

Shares of DIS to buy: 2.692224836924363
Shares of NFLX to buy: 5.073566560291259
Shares of TSLA to buy: 18.37964973187814


- What would have been the average monthly return to the strategy employed by the investor over this time period?
- What would have been the standard deviation of monthly returns to the strategy employed by the investor over this time period?

In [78]:
tickers = ['DIS', 'NFLX', 'TSLA']
start_date = "2016-09-29"
end_date = "2021-09-03"

# Download the data
data = yf.download(tickers, start=start_date, end=end_date)["Adj Close"]


# Define the allocation percentages
dis_pct = 0.25
nflx_pct = 0.5
tsla_pct = 0.25

total_investment = 1000

[*********************100%***********************]  3 of 3 completed


In [79]:
# Resample to monthly frequency
monthly_data = data.resample('M').last()

# Calculate the monthly returns for each stock
returns = monthly_data.pct_change()


In [80]:
# Calculate the portfolio value for each month
first_date = monthly_data.index[0]
dis_price = data.loc[first_date, ("DIS")]
nflx_price = data.loc[first_date, ("NFLX")]
tsla_price = data.loc[first_date, ("TSLA")]
dis_shares = round((dis_pct * 1000) / dis_price)
nflx_shares = round((nflx_pct * 1000) / nflx_price)
tsla_shares = round((tsla_pct * 1000) / tsla_price)
portfolio_value = pd.DataFrame(index=monthly_data.index)
portfolio_value["DIS"] = monthly_data["DIS"] * dis_shares
portfolio_value["NFLX"] = monthly_data["NFLX"] * nflx_shares
portfolio_value["TSLA"] = monthly_data["TSLA"] * tsla_shares
portfolio_value["Total"] = portfolio_value.sum(axis=1)


In [81]:
# Calculate the monthly returns of the portfolio
portfolio_returns = portfolio_value.pct_change()

# Calculate the average monthly return and standard deviation of monthly returns
avg_monthly_return = portfolio_returns["Total"].mean()
std_monthly_return = portfolio_returns["Total"].std()

In [82]:
# Print the results
print("Average monthly return:", round(avg_monthly_return, 4))
print("Standard deviation of monthly returns:", round(std_monthly_return, 4))

Average monthly return: 0.0391
Standard deviation of monthly returns: 0.0954
